<a href="https://colab.research.google.com/github/charlescuvillier/SIM202/blob/main/OPT202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
"""
Spyder Editor

This is a temporary script file.
"""

import numpy as np
#import matplotlib.pyplot as plt
import cvxpy as cp

############   IMPLEMENTATION DES FONCTIONS UTILES    #############

def g(x):
    #x est un vecteur de taille 5, g(x) est un vecteur de taille 3
    y=np.zeros(3)
    y[0] = x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2 + x[4]**2 - 10
    y[1] = x[1]*x[2] - 5*x[4]*x[3]
    y[2] = x[1]**3 - x[0]**3 +1
    return  y

def f(x):
    #x est un vecteur de taille 5, f(x) est un scalaire
    return np.exp(x[0]*x[1]*x[2]*x[3]*x[4]) -0.5*(x[0]**3+x[2]**2+1)**2

def grad_f(x):
    alpha = np.exp(x[0]*x[1]*x[2]*x[3]*x[4])
    g = np.ones(5)*alpha
    for i in range(0,5):
        for j in range(0,5):
            if(j!=i):
                g[i]*=x[j]
    g[0]+= -3*x[0]**2*(x[0]**3+x[1]**2+1)
    g[0]+= -2*x[1]*(x[0]**3+x[1]**2+1)
    return g

def Jacobien_g(x):
    J = np.zeros((3,5))
    for i in range(0,5):
        J[0,i]=2*x[i]
    J[1,1]=x[2]
    J[1,2]=x[1]
    J[1,3]=-5*x[4]
    J[1,4]=-5*x[3]
    J[2,0]=3*x[0]**2
    J[2,0]=3*x[0]**2
    return J


def Hessien_xf(x):
    #x est un vecteur de taille 5
    alpha = np.exp(x[0]*x[1]*x[2]*x[3]*x[4])
    H = np.zeros((5,5))
    #1ere ligne
    H[0,0] = alpha*(x[1]*x[2]*x[3]*x[4])^2  -6*x[0]*(x[0]^3+x[1]^3 +1) -9*x[0]^4
    H[0,1] = x[2]*x[3]*x[4]*(1+x[0]+x[1])*alpha -9*x[0]^2*x[1]^2
    H[0,2] = alpha*(x[1]*x[4]*x[3])*(1+x[0]+x[2])
    H[0,3] = alpha*(x[1]*x[4]*x[2])*(1+x[0]+x[3])
    H[0,4] = alpha*(x[1]*x[2]*x[3])*(1+x[0]+x[4])
    #2e ligne
    H[1,0] = H[0,1]
    H[1,1] = alpha*(x[0]*x[2]*x[3]*x[4])^2  -6*x[1]*(x[0]^3+x[1]^3 +1) -9*x[1]^4
    H[1,2] = alpha*(x[0]*x[4]*x[3])*(1+x[1]+x[2])
    H[1,3] = alpha*(x[0]*x[4]*x[2])*(1+x[1]+x[3])
    H[1,4] = alpha*(x[0]*x[2]*x[3])*(1+x[1]+x[4])
    #3e ligne
    H[2,0] = H[0,2]
    H[2,1] = H[1,2]
    H[2,2] = alpha*(x[1]*x[0]*x[3]*x[4])^2
    H[2,3] = alpha*(x[0]*x[1]*x[3])*(1+x[3]+x[2])
    H[2,4] = alpha*(x[0]*x[1]*x[3])*(1+x[4]+x[2])
    #4e ligne
    H[3,0] = H[0,3]
    H[3,1] = H[1,3]
    H[3,2] = H[2,3]
    H[3,3] = alpha*(x[1]*x[0]*x[2]*x[4])^2
    H[3,4] = alpha*(x[0]*x[1]*x[2])*(1+x[4]+x[3])
    #5e ligne
    H[4,0] = H[0,4]
    H[4,1] = H[1,4]
    H[4,2] = H[2,4]
    H[4,3] = H[3,4]
    H[4,4] = alpha*(x[1]*x[0]*x[2]*x[3])^2
    return H

def Hessien_xlambdag(x,lambd):
    #x est un vecteur de taille 5 et lambd un vecteur de taille 3
    H = 2*lambd[0]*np.eye(5)
    H[1,2]=lambd[1]
    H[2,1]=lambd[1]
    H[3,4]=-5*lambd[1]
    H[4,3]=-5*lambd[1]
    H[0,0]+=6*x[0]*lambd[2]
    H[1,1]+=6*x[1]*lambd[2]
    return H

def Hessien_xLagrangien(x,lambd):
    return Hessien_xf(x) + Hessien_xlambdag(x,lambd)

def grad_xLagrangien(x,lambd):
    return grad_f(x) + np.transpose(Jacobien_g(x))@lambd

def grad_lambdaLagrangien(x,lambd):
    return g(x)



x=np.array([1,1,1,1,1])
x0=np.array([0,0,0,0,0])
lambd=np.array([0,0,1])


########### algorithme ############

#points de depart
x0=np.array([-1.71, 1.59, 1.82, -0.763, -0.763])
x0_tild=np.array([-1.9, 1.82, 2.02, -0.9, -0.9])
x0_chap=np.array([1, 0, 3, 0, 0])


#resolution du probleme par l'algo local SQP
x_k=x0
lambd_k=np.array([1,1,1])
eps=0.01
N=100
k=0
while((np.linalg.norm(grad_xLagrangien(x_k, lambd_k)) > eps)and((np.linalg.norm(grad_lambdaLagrangien(x_k, lambd_k))) > eps) and k<N):
    a = cp.Variable(5)
    objectif = cp.Minimize(0.5*np.transpose(a)@(Hessien_xLagrangien(x_k, lambd_k)@a) + grad_f(x_k)@a)
    contrainte = [g(x_k) + Jacobien_g(x_k)@a ==np.zeros(3)]
    prob = cp.Problem(objectif, contrainte)
    x_k += a.value
    lambd_k = contrainte[0].dual_value 
    k+=1


# #affichage du resultat 
# print("le couple de solution au probleme KKT est (x,lambda) = :" , np.tuple(x_k,lambd_k))
# print("associée a la valeur f(x*)=", f(x_k))
# print("le processus c'est arreté après", k) 
# print("ittérations")

        
            
    
 

TypeError: ignored